In [ ]:
# To be run on Google Colab!
!mkdir -p utoronto/edu/ca/
!wget --directory-prefix=utoronto/edu/ca/  https://raw.githubusercontent.com/rbouadjenek/SIGIR22_LMRec/main/utoronto/edu/ca/data.py   > /dev/null 2> /dev/null 
!wget --directory-prefix=utoronto/edu/ca/ https://raw.githubusercontent.com/rbouadjenek/SIGIR22_LMRec/main/utoronto/edu/ca/models.py  > /dev/null 2> /dev/null 
!wget --directory-prefix=utoronto/edu/ca/ https://raw.githubusercontent.com/rbouadjenek/SIGIR22_LMRec/main/utoronto/edu/ca/utils.py  > /dev/null 2> /dev/null 
!wget --directory-prefix=utoronto/edu/ca/ https://raw.githubusercontent.com/rbouadjenek/SIGIR22_LMRec/main/utoronto/edu/ca/__init__.py  > /dev/null 2> /dev/null 
!wget --directory-prefix=utoronto/edu/ https://raw.githubusercontent.com/rbouadjenek/SIGIR22_LMRec/main/utoronto/edu/ca/__init__.py  > /dev/null 2> /dev/null 
!wget --directory-prefix=utoronto/ https://raw.githubusercontent.com/rbouadjenek/SIGIR22_LMRec/main/utoronto/edu/ca/__init__.py  > /dev/null 2> /dev/null 
!wget --directory-prefix=./ https://raw.githubusercontent.com/rbouadjenek/SIGIR22_LMRec/main/names.txt  > /dev/null 2> /dev/null
!pip install tokenizers transformers > /dev/null 2> /dev/null

from google.colab import drive
drive.mount('/content/drive')


In [ ]:
import tensorflow as tf
from tensorflow import keras
from utoronto.edu.ca.data import Dataset
import utoronto.edu.ca.models as models
import utoronto.edu.ca.utils as utils
import matplotlib.pyplot as plt
import pandas as pd

%matplotlib inline
%load_ext autoreload
%autoreload 2
%load_ext tensorboard


# Collect the data

In [ ]:
dataset = Dataset('atlanta', masking=True)
# dataset = Dataset('austin', masking=True)
# dataset = Dataset('boston', masking=True)
# dataset = Dataset('columbus', masking=True)
# dataset = Dataset('orlando', masking=True)
# dataset = Dataset('portland', masking=True)
# dataset = Dataset('toronto', masking=True)

early_stopping_callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=3)


# Create the model

In [ ]:
# Create the model
model = models.get_model(dataset.max_len, dataset.num_labels, tpu=True)
model.summary()
#train model
history = model.fit(dataset.X_train,
                    dataset.y_train,
                    validation_data=(dataset.X_eval, dataset.y_eval),
                    epochs=15,
                    batch_size=64,
                    callbacks=[early_stopping_callback])
utils.learning_plots(history)


# Save the model

In [ ]:
file_name = './saved_weights_' + dataset.dataset + '.h5'
model.save_weights(file_name, overwrite=True)


In [ ]:
!cp $file_name /content/drive/MyDrive/


# Debug